# Project 6: Analyzing Stock Sentiment from Twits
## Instructions
Each problem consists of a function to implement and instructions on how to implement the function.  The parts of the function that need to be implemented are marked with a `# TODO` comment.

## Packages
When you implement the functions, you'll only need to you use the packages you've used in the classroom, like [Pandas](https://pandas.pydata.org/) and [Numpy](http://www.numpy.org/). These packages will be imported for you. We recommend you don't add any import statements, otherwise the grader might not be able to run your code.

### Load Packages

In [1]:
import json
import nltk
import os
import random
import re
import torch

from torch import nn, optim
import torch.nn.functional as F

from tqdm import tqdm, trange
import pickle
import os.path

## Introduction
When deciding the value of a company, it's important to follow the news. For example, a product recall or natural disaster in a company's product chain. You want to be able to turn this information into a signal. Currently, the best tool for the job is a Neural Network. 

For this project, you'll use posts from the social media site [StockTwits](https://en.wikipedia.org/wiki/StockTwits). The community on StockTwits is full of investors, traders, and entrepreneurs. Each message posted is called a Twit. This is similar to Twitter's version of a post, called a Tweet. You'll build a model around these twits that generate a sentiment score.

We've collected a bunch of twits, then hand labeled the sentiment of each. To capture the degree of sentiment, we'll use a five-point scale: very negative, negative, neutral, positive, very positive. Each twit is labeled -2 to 2 in steps of 1, from very negative to very positive respectively. You'll build a sentiment analysis model that will learn to assign sentiment to twits on its own, using this labeled data.

The first thing we should to do, is load the data.

## Import Twits 
### Load Twits Data 
This JSON file contains a list of objects for each twit in the `'data'` field:

```
{'data':
  {'message_body': 'Neutral twit body text here',
   'sentiment': 0},
  {'message_body': 'Happy twit body text here',
   'sentiment': 1},
   ...
}
```

The fields represent the following:

* `'message_body'`: The text of the twit.
* `'sentiment'`: Sentiment score for the twit, ranges from -2 to 2 in steps of 1, with 0 being neutral.


To see what the data look like by printing the first 10 twits from the list. 

In [2]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'twits.json'), 'r') as f:
    twits = json.load(f)

print(twits['data'][:10])

[{'message_body': '$FITB great buy at 26.00...ill wait', 'sentiment': 2, 'timestamp': '2018-07-01T00:00:09Z'}, {'message_body': '@StockTwits $MSFT', 'sentiment': 1, 'timestamp': '2018-07-01T00:00:42Z'}, {'message_body': '#STAAnalystAlert for $TDG : Jefferies Maintains with a rating of Hold setting target price at USD 350.00. Our own verdict is Buy  http://www.stocktargetadvisor.com/toprating', 'sentiment': 2, 'timestamp': '2018-07-01T00:01:24Z'}, {'message_body': '$AMD I heard there’s a guy who knows someone who thinks somebody knows something - on StockTwits.', 'sentiment': 1, 'timestamp': '2018-07-01T00:01:47Z'}, {'message_body': '$AMD reveal yourself!', 'sentiment': 0, 'timestamp': '2018-07-01T00:02:13Z'}, {'message_body': '$AAPL Why the drop? I warren Buffet taking out his position?', 'sentiment': 1, 'timestamp': '2018-07-01T00:03:10Z'}, {'message_body': '$BA bears have 1 reason on 06-29 to pay more attention https://dividendbot.com?s=BA', 'sentiment': -2, 'timestamp': '2018-07-01T

### Length of Data
Now let's look at the number of twits in dataset. Print the number of twits below.

In [3]:
"""print out the number of twits"""

# TODO Implement
print(len(twits['data']))



1548010


### Split Message Body and Sentiment Score

In [4]:
messages = [twit['message_body'] for twit in twits['data']]
# Since the sentiment scores are discrete, we'll scale the sentiments to 0 to 4 for use in our network
sentiments = [twit['sentiment'] + 2 for twit in twits['data']]

In [5]:
print(sentiments[:10])

[4, 3, 4, 3, 2, 3, 0, 3, 4, 0]


In [6]:
print(messages[:10])

['$FITB great buy at 26.00...ill wait', '@StockTwits $MSFT', '#STAAnalystAlert for $TDG : Jefferies Maintains with a rating of Hold setting target price at USD 350.00. Our own verdict is Buy  http://www.stocktargetadvisor.com/toprating', '$AMD I heard there’s a guy who knows someone who thinks somebody knows something - on StockTwits.', '$AMD reveal yourself!', '$AAPL Why the drop? I warren Buffet taking out his position?', '$BA bears have 1 reason on 06-29 to pay more attention https://dividendbot.com?s=BA', '$BAC ok good we&#39;re not dropping in price over the weekend, lol', '$AMAT - Daily Chart, we need to get back to above 50.', '$GME 3% drop per week after spike... if no news in 3 months, back to 12s... if BO, then bingo... what is the odds?']


## Preprocessing the Data
With our data in hand we need to preprocess our text. These twits are collected by filtering on ticker symbols where these are denoted with a leader $ symbol in the twit itself. For example,

`{'message_body': 'RT @google Our annual look at the year in Google blogging (and beyond) http://t.co/sptHOAh8 $GOOG',
 'sentiment': 0}`

The ticker symbols don't provide information on the sentiment, and they are in every twit, so we should remove them. This twit also has the `@google` username, again not providing sentiment information, so we should also remove it. We also see a URL `http://t.co/sptHOAh8`. Let's remove these too.

The easiest way to remove specific words or phrases is with regex using the `re` module. You can sub out specific patterns with a space:

```python
re.sub(pattern, ' ', text)
```
This will substitute a space with anywhere the pattern matches in the text. Later when we tokenize the text, we'll split appropriately on those spaces.

### Pre-Processing

In [7]:
nltk.download('wordnet')
nltk.download('punkt')

def preprocess(message):
    """
    This function takes a string as input, then performs these operations: 
        - lowercase
        - remove URLs
        - remove ticker symbols 
        - removes punctuation
        - tokenize by splitting the string on whitespace 
        - removes any single character tokens
    
    Parameters
    ----------
        message : The text message to be preprocessed.
        
    Returns
    -------
        tokens: The preprocessed text into tokens.
    """ 
    #TODO: Implement 
    
    # Lowercase the twit message
    text = message.lower()
#     print('\n');print(text);
    # Replace URLs with a space in the message
    text = re.sub(r"http\S+", " ", text)
#     print(text)
    # Replace ticker symbols with a space. The ticker symbols are any stock symbol that starts with $.
    text = re.sub(r"\$\S+", " ", text)
#     print(text)
    # Replace StockTwits usernames with a space. The usernames are any word that starts with @.
    text = re.sub(r"@\S+", " ", text)
#     print(text)
    # Replace everything not a letter with a space
    text = re.sub(r'[^a-zA-Z]', ' ', text)
#     print(text)
    # Tokenize by splitting the string on whitespace into a list of words
    tokens = nltk.tokenize.word_tokenize(text)
#     print(tokens)
    # Lemmatize words using the WordNetLemmatizer. You can ignore any word that is not longer than one character.
    wnl = nltk.stem.WordNetLemmatizer()
    tokens = [wnl.lemmatize(token) for token in tokens if len(token) > 1]
#     print(tokens)
    return tokens

for i in range(10):
    preprocess(messages[i])

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Preprocess All the Twits 
Now we can preprocess each of the twits in our dataset. Apply the function `preprocess` to all the twit messages.

In [8]:
# TODO Implement

tokenized = []
filename = 'tokenized.bin'
if os.path.exists(filename):
    print('Loading from file...')
    with open(filename, 'rb') as fp:
        tokenized = pickle.load(fp)
else:
    print('Processing and saving....')
    tokenized = [preprocess(message) for message in tqdm(messages)]
    with open(filename, 'wb') as fp:
        pickle.dump(tokenized, fp)

print(f'# twits: {len(tokenized)}')
assert len(tokenized) == len(messages)


  0%|          | 411/1548010 [00:00<24:08, 1068.68it/s]

Processing and saving....


100%|██████████| 1548010/1548010 [08:24<00:00, 3069.31it/s]


# twits: 1548010


### Bag of Words
Now with all of our messages tokenized, we want to create a vocabulary and count up how often each word appears in our entire corpus. Use the [`Counter`](https://docs.python.org/3.1/library/collections.html#collections.Counter) function to count up all the tokens.

In [9]:
from collections import Counter


"""
Create a vocabulary by using Bag of words
"""

# TODO: Implement
# collapse list of lists into single list of tokens
stacked_tokens = [word for twit in tqdm(tokenized) for word in twit]
bow = Counter(stacked_tokens)
# print(bow)

100%|██████████| 1548010/1548010 [00:01<00:00, 1087652.00it/s]


In [10]:
bow.most_common(3)

[('the', 398753), ('to', 379487), ('is', 284865)]

### Frequency of Words Appearing in Message
With our vocabulary, now we'll remove some of the most common words such as 'the', 'and', 'it', etc. These words don't contribute to identifying sentiment and are really common, resulting in a lot of noise in our input. If we can filter these out, then our network should have an easier time learning.

We also want to remove really rare words that show up in a only a few twits. Here you'll want to divide the count of each word by the number of messages. Then remove words that only appear in some small fraction of the messages.

In [15]:
"""
Set the following variables:
    freqs
    low_cutoff
    high_cutoff
    K_most_common
"""

# TODO Implement 
common_to_keep = ['up', 'buy','short', 'call', 'sell', 'down', 'out', 'more', 'like', 'good', 'hold', 'long', 'put', 'growth', 'amp', 'bear', 'over', 'amc', 'bmo' , 'big', 'bull']
# Dictionart that contains the Frequency of words appearing in messages.
# The key is the token and the value is the frequency of that word in the corpus.
total_num_words = len(stacked_tokens) #sum(bow.values())
print(f'Starting vocab {total_num_words}')
freqs = {key: value/total_num_words for key, value in bow.items()}

# Float that is the frequency cutoff. Drop words with a frequency that is lower or equal to this number.
low_cutoff = 5e-6

# Integer that is the cut off for most common words. Drop words that are the `high_cutoff` most common words.
high_cutoff = 20

# The k most common words in the corpus. Use `high_cutoff` as the k.
K_most_common = [word[0] for word in bow.most_common(high_cutoff)]
print(f'\nMost common: ({len(K_most_common)}):\n{K_most_common}')
K_most_common = list(set(K_most_common).difference(common_to_keep))

filtered_words = [word for word in freqs if (freqs[word] > low_cutoff and word not in K_most_common)]
print(f'\nMost common w.keep: ({len(K_most_common)}):\n{K_most_common}')
print(f'\nEnding vocab:{len(filtered_words)}') 

Starting vocab 15545714

Most common: (20):
['the', 'to', 'is', 'for', 'on', 'of', 'and', 'in', 'this', 'it', 'at', 'will', 'up', 'are', 'you', 'that', 'be', 'short', 'what', 'today']

Most common w.keep: (18):
['today', 'be', 'what', 'in', 'to', 'is', 'for', 'will', 'the', 'that', 'you', 'and', 'are', 'on', 'this', 'of', 'it', 'at']

Ending vocab:6609


### Updating Vocabulary by Removing Filtered Words
Let's creat three variables that will help with our vocabulary.

In [16]:
"""
Set the following variables:
    vocab
    id2vocab
    filtered
"""

#TODO Implement
def remove_common_tokens(tokenized):
    output = []
        
    for twit in tqdm(tokenized, position=0, leave=True):
        output.append([word for word in twit if word in filtered_words])
    
    return output


# A dictionary for the `filtered_words`. The key is the word and value is an id that represents the word. 
vocab = {word: i for i, word in enumerate(filtered_words)}
# Reverse of the `vocab` dictionary. The key is word id and value is the word. 
id2vocab = {v: k for k, v in vocab.items()}
# tokenized with the words not in `filtered_words` removed.

filename = 'filtered.bin'
if os.path.exists(filename):
    print('Loading from file...')
    with open(filename, 'rb') as fp:
        filtered = pickle.load(fp)
else:
    filtered = remove_common_tokens(tokenized)
    with open(filename, 'wb') as fp:
        pickle.dump(filtered, fp)
        


100%|██████████| 1548010/1548010 [15:16<00:00, 1689.78it/s]


### Balancing the classes
Let's do a few last pre-processing steps. If we look at how our twits are labeled, we'll find that 50% of them are neutral. This means that our network will be 50% accurate just by guessing 0 every single time. To help our network learn appropriately, we'll want to balance our classes.
That is, make sure each of our different sentiment scores show up roughly as frequently in the data.

What we can do here is go through each of our examples and randomly drop twits with neutral sentiment. What should be the probability we drop these twits if we want to get around 20% neutral twits starting at 50% neutral? We should also take this opportunity to remove messages with length 0.

In [17]:
balanced = {'messages': [], 'sentiments':[]}

n_neutral = sum(1 for each in sentiments if each == 2)
N_examples = len(sentiments)
keep_prob = (N_examples - n_neutral)/4/n_neutral

for idx, sentiment in enumerate(sentiments):
    message = filtered[idx]
    if len(message) == 0:
        # skip this message because it has length zero
        continue
    elif sentiment != 2 or random.random() < keep_prob:
        balanced['messages'].append(message)
        balanced['sentiments'].append(sentiment) 

If you did it correctly, you should see the following result 

In [18]:
n_neutral = sum(1 for each in balanced['sentiments'] if each == 2)
N_examples = len(balanced['sentiments'])
n_neutral/N_examples

0.19458456684325334

Finally let's convert our tokens into integer ids which we can pass to the network.

In [22]:
filename = 'processed.bin'
if os.path.exists(filename):
    print('Loading from file...')
    with open(filename, 'rb') as fp:
        processed = pickle.load(fp)
else:
    print('processing and saving...')
    processed = {'token_ids': [[vocab[word] for word in message] for message in tqdm(balanced['messages'], position=0, leave=True)]
                 , 'sentiments': balanced['sentiments']}
    with open(filename, 'wb') as fp:
        pickle.dump(processed, fp)

  5%|▌         | 55727/1026880 [00:00<00:03, 269373.91it/s]

processing and saving...


100%|██████████| 1026880/1026880 [00:05<00:00, 196739.63it/s]


## Neural Network
Now we have our vocabulary which means we can transform our tokens into ids, which are then passed to our network. So, let's define the network now!

Here is a nice diagram showing the network we'd like to build: 

#### Embed -> RNN -> Dense -> Softmax
### Implement the text classifier
Before we build text classifier, if you remember from the other network that you built in  "Sentiment Analysis with an RNN"  exercise  - which there, the network called " SentimentRNN", here we named it "TextClassifer" - consists of three main parts: 1) init function `__init__` 2) forward pass `forward`  3) hidden state `init_hidden`. 

This network is pretty similar to the network you built expect in the  `forward` pass, we use softmax instead of sigmoid. The reason we are not using sigmoid is that the output of NN is not a binary. In our network, sentiment scores have 5 possible outcomes. We are looking for an outcome with the highest probability thus softmax is a better choice.

In [23]:
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_size, lstm_size, output_size, lstm_layers=1, dropout=0.1):
        """
        Initialize the model by setting up the layers.
        
        Parameters
        ----------
            vocab_size : The vocabulary size.
            embed_size : The embedding layer size.
            lstm_size : The LSTM layer size.
            output_size : The output size.
            lstm_layers : The number of LSTM layers.
            dropout : The dropout probability.
        """
        
        super(TextClassifier, self).__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.lstm_size = lstm_size
        self.output_size = output_size
        self.lstm_layers = lstm_layers
        self.dropout = dropout
        
        # TODO Implement

        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, lstm_size, lstm_layers, 
                            dropout=dropout, batch_first=False)

        # dropout layer
        self.dropout = nn.Dropout(0.3)

        # linear and sigmoid layers
        self.fc = nn.Linear(lstm_size, output_size)
        self.log_softmax = nn.LogSoftmax(dim=1)


    def init_hidden(self, batch_size):
        """ 
        Initializes hidden state
        
        Parameters
        ----------
            batch_size : The size of batches.
        
        Returns
        -------
            hidden_state
            
        """
        
        # TODO Implement 
        
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        
        weight = next(self.parameters()).data

        if (torch.cuda.is_available()):
            hidden = (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_().cuda(),
                  weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_().cuda())
        else:
            hidden = (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_(),
                      weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_())

        return hidden


    def forward(self, nn_input, hidden_state):
        """
        Perform a forward pass of our model on nn_input.
        
        Parameters
        ----------
            nn_input : The batch of input to the NN.
            hidden_state : The LSTM hidden state.

        Returns
        -------
            logps: log softmax output
            hidden_state: The new hidden state.

        """
        
        # TODO Implement 
        batch_size = nn_input.size(0)

        # embeddings and lstm_out
        embeds = self.embedding(nn_input)
        lstm_out, hidden = self.lstm(embeds, hidden_state)

        # stack up lstm outputs
#         lstm_out = lstm_out.contiguous().view(-1, self.lstm_size)
        lstm_out = lstm_out[-1]
        
        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)

        # softmax function
        out = self.log_softmax(out)

        # reshape to be batch_size first
#         out = out.view(batch_size, -1)
#         out = out[:, -1] # get last batch of labels

        # return last softmax output and hidden state
        return out, hidden

### View Model

In [24]:
torch.cuda.is_available()

True

In [27]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = TextClassifier(len(vocab), 10, 6, 5, dropout=0.1, lstm_layers=2)
model.embedding.weight.data.uniform_(-1, 1)
input = torch.randint(0, 1000, (5, 4), dtype=torch.int64).to(device)
hidden = model.init_hidden(4)

model.to(device)

logps, _ = model.forward(input, hidden)
print(logps)

tensor([[-1.4642, -1.3641, -1.5077, -1.9847, -1.8689],
        [-1.6588, -1.3323, -1.4141, -1.9463, -1.8339],
        [-1.6441, -1.3390, -1.4065, -1.9637, -1.8365],
        [-1.3224, -1.4369, -1.6437, -2.0600, -1.7422]], device='cuda:0')


## Training
### DataLoaders and Batching
Now we should build a generator that we can use to loop through our data. It'll be more efficient if we can pass our sequences in as batches. Our input tensors should look like `(sequence_length, batch_size)`. So if our sequences are 40 tokens long and we pass in 25 sequences, then we'd have an input size of `(40, 25)`.

If we set our sequence length to 40, what do we do with messages that are more or less than 40 tokens? For messages with fewer than 40 tokens, we will pad the empty spots with zeros. We should be sure to **left** pad so that the RNN starts from nothing before going through the data. If the message has 20 tokens, then the first 20 spots of our 40 long sequence will be 0. If a message has more than 40 tokens, we'll just keep the first 40 tokens.

In [28]:
def dataloader(messages, labels, sequence_length=30, batch_size=32, shuffle=False):
    """ 
    Build a dataloader.
    """
    if shuffle:
        indices = list(range(len(messages)))
        random.shuffle(indices)
        messages = [messages[idx] for idx in indices]
        labels = [labels[idx] for idx in indices]

    total_sequences = len(messages)

    for ii in range(0, total_sequences, batch_size):
        batch_messages = messages[ii: ii+batch_size]
        
        # First initialize a tensor of all zeros
        batch = torch.zeros((sequence_length, len(batch_messages)), dtype=torch.int64)
        for batch_num, tokens in enumerate(batch_messages):
            token_tensor = torch.tensor(tokens)
            # Left pad!
            start_idx = max(sequence_length - len(token_tensor), 0)
            batch[start_idx:, batch_num] = token_tensor[:sequence_length]
        
        label_tensor = torch.tensor(labels[ii: ii+len(batch_messages)])
        
        yield batch, label_tensor

### Training and  Validation
With our data in nice shape, we'll split it into training and validation sets.

In [32]:
"""
Split data into training and validation datasets. Use an appropriate split size.
The features are the `token_ids` and the labels are the `sentiments`.
"""   

# TODO Implement
split_frac = 0.9
features = processed['token_ids']
encoded_labels = processed['sentiments']

## split data into training, validation, and test data (features and labels, x and y)

split_idx = int(len(features)*split_frac)
train_x, remaining_x = features[:split_idx], features[split_idx:]
train_y, remaining_y = encoded_labels[:split_idx], encoded_labels[split_idx:]

# test_idx = int(len(remaining_x)*0.5)
# val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
# val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]

## print out the shapes of your resultant feature data
print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(len(train_x)), 
      "\nValidation set: \t{}".format(len(remaining_x)))#,
#       "\nTest set: \t\t{}".format(test_x.shape))

train_features = train_x
valid_features = remaining_x
train_labels = train_y
valid_labels = remaining_y

			Feature Shapes:
Train set: 		924192 
Validation set: 	102688


In [36]:
text_batch, labels = next(iter(dataloader(train_features, train_labels, sequence_length=20, batch_size=64)))
model = TextClassifier(len(vocab)+1, 200, 128, 5, dropout=0.)
model.to(device)
hidden = model.init_hidden(64)
for each in hidden:
    each.to(device)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
text_batch, labels = text_batch.to(device), labels.to(device)


logps, hidden = model.forward(text_batch, hidden)

### Training
It's time to train the neural network!

In [37]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = TextClassifier(len(vocab)+1, 1024, 512, 5, lstm_layers=2, dropout=0.2)
model.embedding.weight.data.uniform_(-1, 1)
model.to(device)

TextClassifier(
  (embedding): Embedding(6610, 1024)
  (lstm): LSTM(1024, 512, num_layers=2, dropout=0.2)
  (dropout): Dropout(p=0.3)
  (fc): Linear(in_features=512, out_features=5, bias=True)
  (log_softmax): LogSoftmax()
)

In [38]:
"""
Train your model with dropout. Make sure to clip your gradients.
Print the training loss, validation loss, and validation accuracy for every 100 steps.
"""

epochs = 2
batch_size = 512
learning_rate = 1e-2
clip = 5
seq_len = 20

print_every = 100
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
model.train()

for epoch in range(epochs):
    print('Starting epoch {}'.format(epoch + 1))
    hidden = model.init_hidden(batch_size)
    steps = 0
    best_val_acc = 0
    for text_batch, labels in dataloader(
            train_features, train_labels, batch_size=batch_size, sequence_length=seq_len, shuffle=True):
        # handle incomplete batch
        if text_batch.size() != torch.Size([seq_len, batch_size]):
            continue
            
        steps += 1
        hidden = tuple([each.data for each in hidden])
        
        # Set Device
        text_batch, labels = text_batch.to(device), labels.to(device)
        for each in hidden:
            each.to(device)
        
        # TODO Implement: Train Model
        # zero accumulated gradients
        model.zero_grad()

        # get the output from the model
        output, hidden = model(text_batch, hidden)

        # calculate the loss and perform backprop
        loss = criterion(output, labels)
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        if steps % print_every == 0:
            model.eval()
            
            # TODO Implement: Print metrics
            # Get validation loss
            val_h = model.init_hidden(batch_size)
            val_losses = []
            val_accuracy = []

            for inputs, labels in dataloader(
                valid_features, valid_labels, batch_size=batch_size, sequence_length=seq_len):
                # handle incomplete batch
                if inputs.size() != torch.Size([seq_len, batch_size]):
                    continue
                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_h = tuple([each.data for each in val_h])

                inputs, labels = inputs.to(device), labels.to(device)
                for each in val_h:
                    each.to(device)    

                output, val_h = model(inputs, val_h)
                val_loss = criterion(output, labels)

                val_losses.append(val_loss.item())
                
                val_ps = torch.exp(output)
                top_p, top_class = val_ps.topk(1, dim=1)
                val_accuracy.append(torch.mean((top_class == labels.view(*top_class.shape))
                                               .type(torch.FloatTensor)).item())
                this_val_acc = sum(val_accuracy)/len(val_accuracy)
                
            print("Epoch: {}/{}...".format(epoch+1, epochs),
                  "Step: {}...".format(steps),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(sum(val_losses)/len(val_losses)),
                  "Val Accuracy: {:.4f}".format(this_val_acc))
            
            
            if this_val_acc > best_val_acc:
                torch.save({
                    'epoch': epoch,
                    'step': steps,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': loss,
                    }, 'best_model')
                best_val_acc = this_val_acc
                print("Checkpoint - model saved")
            
            model.train()

Starting epoch 1
Epoch: 1/2... Step: 100... Loss: 0.954291... Val Loss: 0.974383 Val Accuracy: 0.6257
Checkpoint - model saved
Epoch: 1/2... Step: 200... Loss: 0.831131... Val Loss: 0.905426 Val Accuracy: 0.6523
Checkpoint - model saved
Epoch: 1/2... Step: 300... Loss: 0.832092... Val Loss: 0.877984 Val Accuracy: 0.6607
Checkpoint - model saved
Epoch: 1/2... Step: 400... Loss: 0.835736... Val Loss: 0.859165 Val Accuracy: 0.6725
Checkpoint - model saved
Epoch: 1/2... Step: 500... Loss: 0.833271... Val Loss: 0.856935 Val Accuracy: 0.6686
Epoch: 1/2... Step: 600... Loss: 0.854158... Val Loss: 0.845340 Val Accuracy: 0.6722
Epoch: 1/2... Step: 700... Loss: 0.801771... Val Loss: 0.842111 Val Accuracy: 0.6798
Checkpoint - model saved
Epoch: 1/2... Step: 800... Loss: 0.801755... Val Loss: 0.845117 Val Accuracy: 0.6733
Epoch: 1/2... Step: 900... Loss: 0.815948... Val Loss: 0.833120 Val Accuracy: 0.6857
Checkpoint - model saved
Epoch: 1/2... Step: 1000... Loss: 0.887599... Val Loss: 0.830161 Val

## Making Predictions
### Prediction 
Okay, now that you have a trained model, try it on some new twits and see if it works appropriately. Remember that for any new text, you'll need to preprocess it first before passing it to the network. Implement the `predict` function to generate the prediction vector from a message.

In [39]:
def predict(text, model, vocab):
    """ 
    Make a prediction on a single sentence.

    Parameters
    ----------
        text : The string to make a prediction on.
        model : The model to use for making the prediction.
        vocab : Dictionary for word to word ids. The key is the word and the value is the word id.

    Returns
    -------
        pred : Prediction vector
    """    
    
    # TODO Implement
    
    tokens = preprocess(text)
    
    # Filter non-vocab words
    tokens = [word for word in tokens if word in filtered_words]
    # Convert words to ids
    tokens = [vocab[word] for word in tokens]
        
    # Adding a batch dimension
    text_input = torch.tensor(tokens).unsqueeze(1).to(device)

    # Get the NN output
    model.to(device)
    hidden  = model.init_hidden(text_input.size(1))
    for each in hidden:
        each.to(device) 
            
    logps, _ = model.forward(text_input, hidden)
    # Take the exponent of the NN output to get a range of 0 to 1 for each label.
    pred = torch.exp(logps)
    
    return pred

In [40]:
print(f'Device: {device}')
text = "Google is working on self driving cars, I'm bullish on $goog"
model.eval()
predict(text, model, vocab)

Device: cuda


tensor([[ 0.0004,  0.0164,  0.0143,  0.7094,  0.2595]], device='cuda:0')

### Questions: What is the prediction of the model? What is the uncertainty of the prediction?
On a sentiment scale from 0 to 4, the most probable rating is 3 reflecting positive sentiment. This prediction has a likelyhood of 71%

Now we have a trained model and we can make predictions. We can use this model to track the sentiments of various stocks by predicting the sentiments of twits as they are coming in. Now we have a stream of twits. For each of those twits, pull out the stocks mentioned in them and keep track of the sentiments. Remember that in the twits, ticker symbols are encoded with a dollar sign as the first character, all caps, and 2-4 letters, like $AAPL. Ideally, you'd want to track the sentiments of the stocks in your universe and use this as a signal in your larger model(s).

## Testing
### Load the Data 

In [41]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'test_twits.json'), 'r') as f:
    test_data = json.load(f)

### Twit Stream

In [42]:
def twit_stream():
    for twit in test_data['data']:
        yield twit

next(twit_stream())

{'message_body': '$JWN has moved -1.69% on 10-31. Check out the movement and peers at  https://dividendbot.com?s=JWN',
 'timestamp': '2018-11-01T00:00:05Z'}

Using the `prediction` function, let's apply it to a stream of twits.

In [43]:
def score_twits(stream, model, vocab, universe):
    """ 
    Given a stream of twits and a universe of tickers, return sentiment scores for tickers in the universe.
    """
    for twit in stream:

        # Get the message text
        text = twit['message_body']
        symbols = re.findall('\$[A-Z]{2,4}', text)
        score = predict(text, model, vocab)

        for symbol in symbols:
            if symbol in universe:
                yield {'symbol': symbol, 'score': score, 'timestamp': twit['timestamp']}

In [45]:
universe = {'$BBRY', '$AAPL', '$AMZN', '$BABA', '$YHOO', '$LQMT', '$FB', '$GOOG', '$BBBY', '$JNUG', '$SBUX', '$MU'}
score_stream = score_twits(twit_stream(), model, vocab, universe)

next(score_stream)

{'symbol': '$AAPL',
 'score': tensor([[ 0.0101,  0.0780,  0.0995,  0.4527,  0.3597]], device='cuda:0'),
 'timestamp': '2018-11-01T00:00:18Z'}

That's it. You have successfully built a model for sentiment analysis! 

## Submission
Now that you're done with the project, it's time to submit it. Click the submit button in the bottom right. One of our reviewers will give you feedback on your project with a pass or not passed grade. You can continue to the next section while you wait for feedback.